In [1]:
import pandas as pd
import numpy as np

# 1. **데이터 설명**
1. pathway_train, pathway_test, pathway_valid : CCLE에서 pathway score 계산 한 데이터
2. ic_train, ic_test, ic_val : 민감군, 저항군 라벨
3. ccle : 유전자 raw data -> 민감군, 저항군 간 유전자 발현량이 가장 큰 유전자 3개를 genes에 저장
4. tcga: 환자 데이터로 약물 민간/저항 라벨 부착 예정

In [3]:
pathway_train = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/CEDIRANIB/train_pathway_score_CEDIRANIB.csv")
pathway_test = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/CEDIRANIB/test_pathway_score_CEDIRANIB.csv")
pathway_valid = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/CEDIRANIB/val_pathway_score_CEDIRANIB.csv")

ic_train = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/CEDIRANIB/ic_train_CEDIRANIB.csv")
ic_test = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/CEDIRANIB/ic_test_CEDIRANIB.csv")
ic_val = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/CEDIRANIB/ic_val_CEDIRANIB.csv")

ccle = pd.read_csv("C:/Users/User/비어플 의료/#_filtered_CCLE_gene_expression.csv")
ccle.index = ccle["Unnamed: 0"]




In [4]:
pathway_train.set_index("SampleID", inplace= True)
pathway_test.set_index("SampleID", inplace= True)
pathway_valid.set_index("SampleID", inplace= True)

ic_train.set_index("Unnamed: 0", inplace=True)
ic_test.set_index("Unnamed: 0", inplace=True)
ic_val.set_index("Unnamed: 0", inplace=True)


In [5]:
genes = ["MAP4K1", "WAS", "SELPLG"  ]

# 2. **전처리**
1. pathway score scale
2. pathway score에 대해 PCA 진행 후 90% 설명력까지 사용
3. train 기준으로 fjt하고 trest, valid, TCGA에는 transform만 해주기
4. ccle, tcga 유전자들도 sacle해주기
5. ic라벨 인코딩`

In [7]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 스케일링 (z-score 정규화가 되어있더라도 PCA 전에 한 번 더 정규화)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(pathway_train)
X_test_scaled = scaler.transform(pathway_test)
X_val_scaled = scaler.transform(pathway_valid)
# PCA 수행 (주성분 2개로 축소)
pca = PCA(n_components=0.90)
pathway_pca_train = pca.fit_transform(X_scaled)
pathway_pca_test  = pca.transform(X_test_scaled)
pathway_pca_valid = pca.transform(X_val_scaled)

In [8]:
pca_columns = [f"PC{i+1}" for i in range(pathway_pca_train.shape[1])]
pca_train_df_all = pd.DataFrame(pathway_pca_train, columns=pca_columns, index=pathway_train.index)
pca_train_df_all

pca_valid_df_all = pd.DataFrame(pathway_pca_valid, columns=pca_columns, index=pathway_valid.index)
pca_valid_df_all

pca_test_df_all = pd.DataFrame(pathway_pca_test, columns=pca_columns, index=pathway_test.index)
pca_test_df_all

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45,PC46
SampleID,,,,,,,,,,,,,,,,,,,,,
ACH-000823,-8.857699,-2.215490,-1.120246,-0.823611,-2.373167,3.274626,1.038775,1.594026,-1.734462,1.363545,...,1.712137,-0.930890,-0.035250,1.533580,-0.029038,-0.598268,-0.401183,-0.856598,0.159499,0.205002
ACH-000678,-4.497151,0.492021,3.379773,-1.874305,-2.465482,2.585348,-2.057702,-3.528870,-0.733233,-1.958185,...,0.408433,-0.995224,-0.788000,0.127260,-0.153884,-0.747858,1.011083,1.891004,-0.379655,-0.400099
ACH-000138,-4.417090,2.944893,5.817551,-1.096988,-1.538579,-0.870846,1.081872,-0.071248,-1.305760,-1.948108,...,-0.828067,-0.468337,-0.260257,0.397634,-0.950010,1.264251,0.540983,0.409328,-0.821282,-1.221825
ACH-000704,9.330632,-5.256672,0.135887,0.666500,1.387105,-1.339309,1.075058,0.999341,4.313895,-1.794546,...,1.338468,1.139595,1.021956,-1.543697,-0.587347,1.460155,0.800201,-0.339941,0.971586,-0.702930
ACH-000276,6.133993,-5.420277,-1.862820,-2.748638,-0.719810,0.642280,-0.138287,-0.494809,2.004035,-2.768460,...,0.562111,-0.099183,0.691991,1.237861,-1.001297,0.433648,-0.784054,-1.632614,0.660434,-0.335285
ACH-000035,9.294253,-1.839145,0.850174,-3.068360,1.075761,0.446220,-0.881635,1.537264,3.887177,0.424840,...,-0.245553,0.550415,-0.284668,-0.948590,0.849328,0.094666,0.233010,-0.216325,-0.923923,1.483232
ACH-000223,7.136013,-3.612972,4.283164,-1.643811,-0.267390,1.908344,0.868855,-0.541992,2.528907,0.125842,...,1.326951,0.101863,-0.488698,-0.130309,1.028027,0.341148,0.451228,-0.287473,1.197865,0.547659
ACH-000971,2.304680,-4.765937,-2.286625,4.479442,-3.679099,0.214857,0.161667,0.495775,0.980003,1.824361,...,0.446087,-0.747276,0.410122,0.116675,0.254716,-0.717207,-1.269538,-0.271443,0.646169,0.384717
ACH-000858,-20.969666,0.239734,0.705437,0.643144,-1.972278,0.738154,1.875188,-0.306126,-3.095526,-0.184940,...,-0.019214,-1.795866,0.961185,0.520663,0.152835,-0.805263,0.860529,0.651098,-0.071421,1.004690


In [9]:
ccle = ccle[genes]
ccle_expr_train = ccle.loc[pathway_train.index]
ccle_log_train = np.log2(ccle_expr_train+1)

scaler = StandardScaler()
ccle_scaled_train = pd.DataFrame(scaler.fit_transform(ccle_log_train),
                          columns = ccle_expr_train.columns,
                          index = ccle_expr_train.index)

ccle_expr_valid = ccle.loc[pathway_valid.index]
ccle_log_valid = np.log2(ccle_expr_valid+1)

ccle_scaled_valid = pd.DataFrame(scaler.transform(ccle_log_valid),
                          columns = ccle_expr_valid.columns,
                          index = ccle_expr_valid.index)

ccle_expr_test = ccle.loc[pathway_test.index]
ccle_log_test = np.log2(ccle_expr_test+1)
ccle_scaled_test = pd.DataFrame(scaler.transform(ccle_log_test),
                          columns = ccle_expr_test.columns,
                          index = ccle_expr_test.index)

In [10]:
X_train = pd.concat([pca_train_df_all, ccle_scaled_train], axis = 1)
X_valid = pd.concat([pca_valid_df_all, ccle_scaled_valid], axis = 1)
X_test = pd.concat([pca_test_df_all, ccle_scaled_test], axis = 1)

final_scaler = StandardScaler()
X_train_final = final_scaler.fit_transform(X_train)
X_valid_final = final_scaler.transform(X_valid)
X_test_final = final_scaler.transform(X_test)


In [11]:
# 숫자 라벨로 변환 (sensitive = 1, resistant = 0)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(ic_train.iloc[:,0])
y_valid = le.fit_transform(ic_val.iloc[:,0])
y_test = le.fit_transform(ic_test.iloc[:,0])

# **3. 모델링**
1. 로지스틱, 랜덤포레스트, SVM, KNN으로 진행
2. 전처리를 R로 해야했기에 pipeline으로 묶을 수 없어 cross validation이 힘듬
3. 평가지표는 AUC, F1 score 중점으로 사용
4. 수동으로 튜닝 후 성능이 제일 좋은 모델 선택 후 TCGA 라벨 예측

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

In [14]:
## Logistic Regression
lr = LogisticRegression(C=0.01, max_iter=10000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
y_proba_lr = lr.predict_proba(X_test)[:, 1]

## Random Forest
rf = RandomForestClassifier(n_estimators=250, max_depth=5, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

## Support Vector Machine (SVM)
svm = SVC(probability=True, random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
y_proba_svm = svm.predict_proba(X_test)[:, 1]

## K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
y_proba_knn = knn.predict_proba(X_test)[:, 1]

# ── AdaBoost 추가 ──
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(
    n_estimators=50,
    random_state=42
)
ada.fit(X_train_final, y_train)

y_pred_ada   = ada.predict(X_test_final)
y_proba_ada  = ada.predict_proba(X_test_final)[:, 1]

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [15]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

print("===== Validation Set Evaluation =====")

# Logistic Regression - Validation
y_pred_lr_valid = lr.predict(X_valid)
y_proba_lr_valid = lr.predict_proba(X_valid)[:, 1]
print("\n--- Logistic Regression ---")
print(classification_report(y_valid, y_pred_lr_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_lr_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_lr_valid))

# Random Forest - Validation
y_pred_rf_valid = rf.predict(X_valid)
y_proba_rf_valid = rf.predict_proba(X_valid)[:, 1]
print("\n--- Random Forest ---")
print(classification_report(y_valid, y_pred_rf_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_rf_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_rf_valid))

# Support Vector Machine - Validation
y_pred_svm_valid = svm.predict(X_valid)
y_proba_svm_valid = svm.predict_proba(X_valid)[:, 1]
print("\n--- Support Vector Machine ---")
print(classification_report(y_valid, y_pred_svm_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_svm_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_svm_valid))

# K-Nearest Neighbors - Validation
y_pred_knn_valid = knn.predict(X_valid)
y_proba_knn_valid = knn.predict_proba(X_valid)[:, 1]
print("\n--- K-Nearest Neighbors ---")
print(classification_report(y_valid, y_pred_knn_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_knn_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_knn_valid))

# AdaBoost - Validation
y_pred_ada_valid   = ada.predict(X_valid_final)
y_proba_ada_valid  = ada.predict_proba(X_valid_final)[:, 1]
print("\n--- AdaBoost ---")
print(classification_report(y_valid, y_pred_ada_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_ada_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_ada_valid))


===== Validation Set Evaluation =====

--- Logistic Regression ---
              precision    recall  f1-score   support

   resistant       0.53      0.62      0.57        26
   sensitive       0.52      0.44      0.48        25

    accuracy                           0.53        51
   macro avg       0.53      0.53      0.52        51
weighted avg       0.53      0.53      0.53        51

ROC-AUC: 0.5446153846153846
Confusion matrix:
 [[16 10]
 [14 11]]

--- Random Forest ---
              precision    recall  f1-score   support

   resistant       0.53      0.62      0.57        26
   sensitive       0.52      0.44      0.48        25

    accuracy                           0.53        51
   macro avg       0.53      0.53      0.52        51
weighted avg       0.53      0.53      0.53        51

ROC-AUC: 0.48307692307692307
Confusion matrix:
 [[16 10]
 [14 11]]

--- Support Vector Machine ---
              precision    recall  f1-score   support

   resistant       0.48      0.50   

In [16]:
from sklearn.ensemble import VotingClassifier
# ── Soft Voting Ensemble ──
voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('svm', svm),
        ('knn', knn),
        ('ada', ada)
    ],
    voting='soft'
)
voting_clf.fit(X_train_final, y_train)

y_pred_voting  = voting_clf.predict(X_test_final)
y_proba_voting = voting_clf.predict_proba(X_test_final)[:, 1]

print("\n===== Soft Voting Ensemble =====")
print(classification_report(y_test, y_pred_voting, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_voting))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_voting))





===== Soft Voting Ensemble =====
              precision    recall  f1-score   support

   resistant       0.53      0.62      0.57        26
   sensitive       0.52      0.44      0.48        25

    accuracy                           0.53        51
   macro avg       0.53      0.53      0.52        51
weighted avg       0.53      0.53      0.53        51

ROC-AUC: 0.6
Confusion matrix:
 [[16 10]
 [14 11]]


C:\Users\USER\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
